# Test: Brain Connectivity Plot V3 Features

This notebook tests all new features in `create_brain_connectivity_plot`:
1. **Edge width scaling** - Edges scale by absolute matrix value
2. **Vector node sizes** - Node sizes can be an array/vector
3. **Edge file support** - Can load .edge files directly
4. **Node metrics hover** - Display metrics on node hover
5. **Hide nodes with hidden edges** - Nodes toggle with their edge type
6. **Static image export** - Export to PNG, SVG, PDF formats
7. **Export display controls** - Control title/legend visibility in exports
8. **Node visibility fix** - Nodes with both edge types hide when all edges hidden

In [1]:
import sys
import numpy as np
import pandas as pd

# Add package to path
sys.path.insert(0, r'C:\Users\Azad Azargushasb\Research')

from HarrisLabPlotting import (
    create_brain_connectivity_plot,
    load_mesh_file,
    load_node_file,
    load_edge_file,
    convert_node_size_input,
    load_connectivity_input,
    load_node_metrics,
    map_coordinate
)

print("Imports successful!")

Imports successful!


## Test 1: Basic functionality with edge file and positive/negative edge toggling

Using node/edge files to test positive and negative edge toggling.

In [2]:
# File paths for Test 1
node_file = r"C:\Users\Azad Azargushasb\Research\HarrisLabPlotting\test_files\node and edges\total.node"
edge_file = r"C:\Users\Azad Azargushasb\Research\HarrisLabPlotting\test_files\node and edges\total(1).edge"
roi_coords_file = r"C:\Users\Azad Azargushasb\Research\roi_coordinates\atlas_170_coordinates\atlas_170_coordinates_comma.csv"
mesh_file = r"C:\Users\Azad Azargushasb\Research\brain_filled_3_smoothed.gii"

print("Test 1 file paths set.")

Test 1 file paths set.


In [3]:
# Load mesh
print("Loading mesh...")
vertices, faces = load_mesh_file(mesh_file)
print(f"Mesh: {vertices.shape[0]} vertices, {faces.shape[0]} faces")

Loading mesh...
Loading mesh from: C:\Users\Azad Azargushasb\Research\brain_filled_3_smoothed.gii
  Detected GIFTI format (.gii)
  Found faces array with shape: (18857, 3)
  Found vertices array with shape: (9510, 3)
  Successfully loaded mesh: 9510 vertices, 18857 faces
Mesh: 9510 vertices, 18857 faces


In [4]:
# Load ROI coordinates
print("Loading ROI coordinates...")
roi_coords_df = pd.read_csv(roi_coords_file)
print(f"ROI coordinates: {len(roi_coords_df)} ROIs")
print(f"Columns: {roi_coords_df.columns.tolist()}")

Loading ROI coordinates...
ROI coordinates: 170 ROIs
Columns: ['roi_index', 'roi_name', 'cog_x', 'cog_y', 'cog_z', 'cog_voxel_i', 'cog_voxel_j', 'cog_voxel_k']


In [5]:
# Load node file to see its structure
print("Loading node file...")
node_df = load_node_file(node_file)
print(f"Node file: {len(node_df)} nodes")
print(node_df.head())

Loading node file...
Node file: 28 nodes
         x        y        z  size  color   roi_name
0 -57.5089   3.3815  12.8073     4      1   AUD_left
1 -47.7442   2.7895  25.3957     4      1  PtPD_left
2 -19.4928 -15.5129  30.9181     4      1   RSD_left
3  -5.7723   9.5417  27.9448     4      1  RSGc_left
4 -55.3657  10.5549  15.3168     4      1    S1_left


In [6]:
# Load edge file to see its structure  
print("Loading edge file...")
edge_matrix = load_edge_file(edge_file)
print(f"Edge matrix shape: {edge_matrix.shape}")
print(f"Non-zero values: {np.count_nonzero(edge_matrix)}")
print(f"Positive values: {np.sum(edge_matrix > 0)}")
print(f"Negative values: {np.sum(edge_matrix < 0)}")
print(f"Min: {edge_matrix.min():.4f}, Max: {edge_matrix.max():.4f}")

Loading edge file...
Edge matrix shape: (28, 28)
Non-zero values: 54
Positive values: 36
Negative values: 18
Min: -0.6249, Max: 0.9041


In [7]:
# Check if we need to pad/match dimensions
# Edge matrix is node_count x node_count, ROI coords may be different
print(f"Edge matrix: {edge_matrix.shape[0]} x {edge_matrix.shape[1]}")
print(f"ROI coords: {len(roi_coords_df)} ROIs")

# If they don't match, we need to create a matching subset or expand
if edge_matrix.shape[0] != len(roi_coords_df):
    print(f"\nDimension mismatch! Using only first {edge_matrix.shape[0]} ROIs.")
    roi_coords_subset = roi_coords_df.iloc[:edge_matrix.shape[0]].reset_index(drop=True)
else:
    roi_coords_subset = roi_coords_df

Edge matrix: 28 x 28
ROI coords: 170 ROIs

Dimension mismatch! Using only first 28 ROIs.


THis logic is wrong above, to use subset use mapping function from roi coordinates down below. 

In [8]:
node_list =load_node_file(node_file)

In [9]:
node_list

,x,y,z,size,color,roi_name
0,-57.5089,3.3815,12.8073,4,1,AUD_left
1,-47.7442,2.7895,25.3957,4,1,PtPD_left
2,-19.4928,-15.5129,30.9181,4,1,RSD_left
3,-5.7723,9.5417,27.9448,4,1,RSGc_left
4,-55.3657,10.5549,15.3168,4,1,S1_left
5,-32.3096,39.3905,-7.1604,4,1,SaM_left
6,-42.7940,28.4720,-9.6907,4,1,SaP_left
7,-36.1726,-5.1345,29.6702,4,1,V1_left
8,-54.4773,-18.1355,19.8560,4,1,V2L_left
9,-22.8309,-4.3162,30.3309,4,1,V2ML_left


In [10]:
new_roi_coords, unmapped_rois = map_coordinate(roi_coords_df, node_list)

Mapping coordinates from:      roi_index       roi_name      cog_x      cog_y      cog_z  cog_voxel_i  \
0            1  Acumbens_left -15.355556  66.017901 -20.083333    79.284444   
1            2       AID_left -45.765769  64.444362  -7.542770   103.612615   
2            3       AIP_left -61.894410  29.911610 -21.016484   116.515528   
3            4       AIV_left -38.613395  69.366538  -9.036509    97.890716   
4            5  Amygdala_left -40.170219  15.355125 -37.531890    99.136175   
..         ...            ...        ...        ...        ...          ...   
165        166     V2ML_right  22.269830  -6.506728  43.213527    49.184136   
166        167     V2MM_right  16.671175  -6.066461  43.274730    53.663060   
167        168    VIEnt_right  56.357060 -23.136574 -28.778935    21.914352   
168        169   VP_mfb_right  20.854541  50.425775 -34.276781    50.316368   
169        170      VTA_right   2.793561  -5.000000 -29.498106    64.765152   

     cog_voxel_j  cog_vox

In [11]:
# Test 1: Basic plot with edge file - tests positive/negative edge toggling
print("\n" + "="*60)
print("TEST 1: Edge file input + positive/negative edge toggling")
print("="*60)

fig1, stats1 = create_brain_connectivity_plot(
    vertices=vertices,
    faces=faces,
    roi_coords_df=new_roi_coords,
    connectivity_matrix=edge_file,  # Pass edge file path directly!
    plot_title="Test 1: Edge File + Pos/Neg Toggling",
    save_path=r"C:\Users\Azad Azargushasb\Research\HarrisLabPlotting\test_files\test1_edge_file_toggle.html",
    edge_width=(1.0, 10),  # Scaled edge width
    camera_view='superior',
    hide_nodes_with_hidden_edges=True,  # Nodes hide when their edges hide
    node_size=15,
    label_font_size=20
    
)

print("\nGraph Statistics:")
for key, value in stats1.items():
    print(f"  {key}: {value}")


TEST 1: Edge file input + positive/negative edge toggling
Creating brain connectivity visualization: Test 1: Edge File + Pos/Neg Toggling
Connectivity matrix shape: (28, 28)
Node sizes: min=15.0, max=15.0
Added 28 nodes with valid coordinates
Added 27 edges above threshold 0.0
Node classification: 18 pos-only, 7 neg-only, 3 both
Saved interactive visualization to: C:\Users\Azad Azargushasb\Research\HarrisLabPlotting\test_files\test1_edge_file_toggle.html

Graph Statistics:
  total_nodes: 28
  total_edges: 27
  connected_nodes: 28
  isolated_nodes: 0
  positive_edges: 18
  negative_edges: 9
  nodes_with_pos_only: 18
  nodes_with_neg_only: 7
  nodes_with_both: 3
  network_density: 0.07142857142857142
  average_degree: 1.9285714285714286
  edge_width_scaled: True
  node_size_is_vector: False
  node_color_is_vector: False
  module_color_map: None
  top_connected_nodes: [('V1_left', 5), ('SaA_right', 4), ('SaM_right', 4), ('PtPD_left', 3), ('AIP_right', 3), ('Thalamus_A_right', 3), ('Thala

In [12]:
# Display figure
fig1.show()

## Test 2: Node size as vector + Node metrics hover

Using the k5 state data with metrics.

In [13]:
# File paths for Test 2
metrics_file = r"C:\Users\Azad Azargushasb\Research\HarrisLabPlotting\test_files\k5_data\state_0\combined_metrics.csv"
conn_matrix_file = r"C:\Users\Azad Azargushasb\Research\HarrisLabPlotting\test_files\k5_data\state_0\connectivity_matrix.csv"
roi_coords_114 = r"G:\My Drive\research stim data cci\atlas_114_mapped_comma.csv"

print("Test 2 file paths set.")

Test 2 file paths set.


In [14]:
# Load metrics
print("Loading metrics...")
metrics_df = pd.read_csv(metrics_file)
print(f"Metrics: {len(metrics_df)} rows, columns: {metrics_df.columns.tolist()}")
metrics_df.head()

Loading metrics...
Metrics: 114 rows, columns: ['roi_index', 'node_idx', 'roi_name', 'module', 'participation_coef', 'within_module_zscore', 'node_role', 'pc_z_score', 'pc_p_value', 'pc_significant']


,roi_index,node_idx,roi_name,module,participation_coef,within_module_zscore,node_role,pc_z_score,pc_p_value,pc_significant
0,0,0,Acumbens_left,1,0.424765,-0.579078,2,-4.221807,0.006,True
1,1,1,AID_left,2,0.000000,1.991235,1,-13.853918,0.000,True
2,2,2,AIP_left,2,0.196518,0.362086,2,-8.409042,0.000,True
3,3,3,AIV_left,2,0.000000,-0.282568,1,-6.610655,0.001,True
4,4,4,Amygdala_left,2,0.635951,-1.949220,3,0.263637,0.548,False


In [15]:
# Load connectivity matrix
print("Loading connectivity matrix...")
conn_matrix = pd.read_csv(conn_matrix_file, header=None).values
print(f"Connectivity matrix shape: {conn_matrix.shape}")
print(f"Non-zero: {np.count_nonzero(conn_matrix)}")
print(f"Min: {conn_matrix.min():.4f}, Max: {conn_matrix.max():.4f}")

Loading connectivity matrix...
Connectivity matrix shape: (114, 114)
Non-zero: 904
Min: 0.0000, Max: 0.6060


In [16]:
# Load ROI coordinates for 114 atlas
print("Loading 114 ROI coordinates...")
roi_coords_114_df = pd.read_csv(roi_coords_114)
print(f"ROI coords: {len(roi_coords_114_df)} ROIs")

Loading 114 ROI coordinates...
ROI coords: 114 ROIs


In [17]:
# Create node sizes from participation coefficient
# Scale PC values to reasonable node sizes (e.g., 5-20)
pc_values = metrics_df['participation_coef'].values
node_sizes_vector = 5 + (pc_values / pc_values.max()) * 15  # Scale to 5-20

print(f"Node sizes from PC: min={node_sizes_vector.min():.1f}, max={node_sizes_vector.max():.1f}")

Node sizes from PC: min=5.0, max=20.0


In [18]:
# Test 2: Vector node sizes + metrics hover
print("\n" + "="*60)
print("TEST 2: Vector node sizes + Metrics hover")
print("="*60)

fig2, stats2 = create_brain_connectivity_plot(
    vertices=vertices,
    faces=faces,
    roi_coords_df=roi_coords_114_df,
    connectivity_matrix=conn_matrix,
    plot_title="Test 2: Vector Node Sizes + Metrics Hover",
    save_path=r"C:\Users\Azad Azargushasb\Research\HarrisLabPlotting\test_files\test2_vector_sizes_metrics.html",
    node_size=node_sizes_vector,  # Vector of node sizes!
    edge_width=(0.5, 4.0),  # Scaled edge width
    camera_view='oblique',
    node_metrics=metrics_df,  # Metrics for hover!
    hide_nodes_with_hidden_edges=True
)

print("\nGraph Statistics:")
for key, value in stats2.items():
    print(f"  {key}: {value}")


TEST 2: Vector node sizes + Metrics hover
Creating brain connectivity visualization: Test 2: Vector Node Sizes + Metrics Hover
Connectivity matrix shape: (114, 114)
Node sizes: min=5.0, max=20.0
Loaded node metrics with columns: ['roi_index', 'node_idx', 'roi_name', 'module', 'participation_coef', 'within_module_zscore', 'node_role', 'pc_z_score', 'pc_p_value', 'pc_significant']
Added 114 nodes with valid coordinates
Added 452 edges above threshold 0.0
Node classification: 114 pos-only, 0 neg-only, 0 both
Saved interactive visualization to: C:\Users\Azad Azargushasb\Research\HarrisLabPlotting\test_files\test2_vector_sizes_metrics.html

Graph Statistics:
  total_nodes: 114
  total_edges: 452
  connected_nodes: 114
  isolated_nodes: 0
  positive_edges: 452
  negative_edges: 0
  nodes_with_pos_only: 114
  nodes_with_neg_only: 0
  nodes_with_both: 0
  network_density: 0.07017543859649122
  average_degree: 7.9298245614035086
  edge_width_scaled: True
  node_size_is_vector: True
  node_colo

In [19]:
# Display figure
fig2.show()

## Test 3: Test utility functions directly

In [20]:
# Test convert_node_size_input with different inputs
print("Testing convert_node_size_input...")

n_nodes = 10

# Scalar
sizes_scalar = convert_node_size_input(8, n_nodes)
print(f"Scalar input: {sizes_scalar}")

# List
sizes_list = convert_node_size_input([1,2,3,4,5,6,7,8,9,10], n_nodes)
print(f"List input: {sizes_list}")

# Numpy array
sizes_np = convert_node_size_input(np.arange(1, 11), n_nodes)
print(f"Numpy input: {sizes_np}")

# Dictionary
sizes_dict = convert_node_size_input({0: 15, 5: 20, 9: 25}, n_nodes, default_size=5)
print(f"Dict input: {sizes_dict}")

# Pandas Series
sizes_series = convert_node_size_input(pd.Series(range(10, 20)), n_nodes)
print(f"Series input: {sizes_series}")

Testing convert_node_size_input...
Scalar input: [8. 8. 8. 8. 8. 8. 8. 8. 8. 8.]
List input: [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
Numpy input: [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
Dict input: [15  5  5  5  5 20  5  5  5 25]
Series input: [10. 11. 12. 13. 14. 15. 16. 17. 18. 19.]


In [21]:
# Test load_connectivity_input with different inputs
print("Testing load_connectivity_input...")

# From numpy array
test_matrix = np.random.rand(10, 10)
loaded1 = load_connectivity_input(test_matrix)
print(f"Numpy array: {loaded1.shape}")

# From CSV file
loaded2 = load_connectivity_input(conn_matrix_file)
print(f"CSV file: {loaded2.shape}")

# From edge file
loaded3 = load_connectivity_input(edge_file)
print(f"Edge file: {loaded3.shape}")

Testing load_connectivity_input...
Numpy array: (10, 10)
CSV file: (114, 114)
Edge file: (28, 28)


In [22]:
# Test load_node_metrics
print("Testing load_node_metrics...")

metrics = load_node_metrics(metrics_file)
print(f"Loaded metrics: {len(metrics)} rows, {len(metrics.columns)} columns")
print(f"Columns: {metrics.columns.tolist()}")

Testing load_node_metrics...
Loaded metrics: 114 rows, 10 columns
Columns: ['roi_index', 'node_idx', 'roi_name', 'module', 'participation_coef', 'within_module_zscore', 'node_role', 'pc_z_score', 'pc_p_value', 'pc_significant']


## Test 4: Fixed edge width (no scaling)

In [23]:
# Test 4: Fixed edge width
print("\n" + "="*60)
print("TEST 4: Fixed edge width (no scaling)")
print("="*60)

fig4, stats4 = create_brain_connectivity_plot(
    vertices=vertices,
    faces=faces,
    roi_coords_df=roi_coords_114_df,
    connectivity_matrix=conn_matrix,
    plot_title="Test 4: Fixed Edge Width",
    save_path=r"C:\Users\Azad Azargushasb\Research\HarrisLabPlotting\test_files\test4_fixed_edge_width.html",
    node_size=10,  # Scalar
    edge_width=2.0,  # Fixed width (not a tuple)
    camera_view='anterior'
)

print(f"Edge width scaled: {stats4['edge_width_scaled']}")
print(f"Node size is vector: {stats4['node_size_is_vector']}")


TEST 4: Fixed edge width (no scaling)
Creating brain connectivity visualization: Test 4: Fixed Edge Width
Connectivity matrix shape: (114, 114)
Node sizes: min=10.0, max=10.0
Added 114 nodes with valid coordinates
Added 452 edges above threshold 0.0
Node classification: 114 pos-only, 0 neg-only, 0 both
Saved interactive visualization to: C:\Users\Azad Azargushasb\Research\HarrisLabPlotting\test_files\test4_fixed_edge_width.html
Edge width scaled: False
Node size is vector: False


## Test 5: Static Image Export (PNG, SVG, PDF)

Test the new `export_image` parameter for generating publication-quality static images.

**IMPORTANT:** Run the cell below first to reload the module with the latest changes!

In [24]:
# IMPORTANT: Reload the module to get the latest export fixes
import importlib
import HarrisLabPlotting
import HarrisLabPlotting.connectivity

# Reload the connectivity module
importlib.reload(HarrisLabPlotting.connectivity)

# Re-import the function with the latest code
from HarrisLabPlotting.connectivity import create_brain_connectivity_plot

print("Module reloaded successfully!")

Module reloaded successfully!


In [25]:
# Test 5a: Export as PNG (300 DPI - publication quality)
print("\n" + "="*60)
print("TEST 5a: Export as PNG (300 DPI)")
print("="*60)

fig5a, stats5a = create_brain_connectivity_plot(
    vertices=vertices,
    faces=faces,
    roi_coords_df=roi_coords_114_df,
    connectivity_matrix=conn_matrix,
    plot_title="Test 5a: PNG Export (300 DPI)",
    save_path=r"C:\Users\Azad Azargushasb\Research\HarrisLabPlotting\test_files\test5a_png_export.html",
    node_size=node_sizes_vector,
    edge_width=(0.5, 4.0),
    camera_view='superior',
    node_metrics=metrics_df,
    # NEW: Export as PNG
    export_image=r"C:\Users\Azad Azargushasb\Research\HarrisLabPlotting\test_files\test5a_brain_network.png",
    image_dpi=300
)

print("\nPNG export completed!")


TEST 5a: Export as PNG (300 DPI)
Creating brain connectivity visualization: Test 5a: PNG Export (300 DPI)
Connectivity matrix shape: (114, 114)
Node sizes: min=5.0, max=20.0
Loaded node metrics with columns: ['roi_index', 'node_idx', 'roi_name', 'module', 'participation_coef', 'within_module_zscore', 'node_role', 'pc_z_score', 'pc_p_value', 'pc_significant']
Added 114 nodes with valid coordinates
Added 452 edges above threshold 0.0
Node classification: 114 pos-only, 0 neg-only, 0 both
Saved interactive visualization to: C:\Users\Azad Azargushasb\Research\HarrisLabPlotting\test_files\test5a_png_export.html
Note: Scale capped at 4x (effective ~288 DPI) to avoid memory issues
Exporting PNG image...
Exported static image to: C:\Users\Azad Azargushasb\Research\HarrisLabPlotting\test_files\test5a_brain_network.png
  Format: PNG, Size: 2037.3 KB
  Dimensions: 4800x3600 pixels

PNG export completed!


In [26]:
# Test 5b: Export as SVG (vector format - infinitely scalable)
print("\n" + "="*60)
print("TEST 5b: Export as SVG (vector format)")
print("="*60)

fig5b, stats5b = create_brain_connectivity_plot(
    vertices=vertices,
    faces=faces,
    roi_coords_df=roi_coords_114_df,
    connectivity_matrix=conn_matrix,
    plot_title="Test 5b: SVG Export (Vector)",
    save_path=r"C:\Users\Azad Azargushasb\Research\HarrisLabPlotting\test_files\test5b_svg_export.html",
    node_size=node_sizes_vector,
    edge_width=(0.5, 4.0),
    camera_view='oblique',
    node_metrics=metrics_df,
    # NEW: Export as SVG
    export_image=r"C:\Users\Azad Azargushasb\Research\HarrisLabPlotting\test_files\test5b_brain_network.svg"
)

print("\nSVG export completed!")


TEST 5b: Export as SVG (vector format)
Creating brain connectivity visualization: Test 5b: SVG Export (Vector)
Connectivity matrix shape: (114, 114)
Node sizes: min=5.0, max=20.0
Loaded node metrics with columns: ['roi_index', 'node_idx', 'roi_name', 'module', 'participation_coef', 'within_module_zscore', 'node_role', 'pc_z_score', 'pc_p_value', 'pc_significant']
Added 114 nodes with valid coordinates
Added 452 edges above threshold 0.0
Node classification: 114 pos-only, 0 neg-only, 0 both
Saved interactive visualization to: C:\Users\Azad Azargushasb\Research\HarrisLabPlotting\test_files\test5b_svg_export.html
Exporting SVG image...
Exported static image to: C:\Users\Azad Azargushasb\Research\HarrisLabPlotting\test_files\test5b_brain_network.svg
  Format: SVG, Size: 507.0 KB

SVG export completed!


In [27]:
# Test 5c: Export as PDF (vector format for publications)
print("\n" + "="*60)
print("TEST 5c: Export as PDF (publication format)")
print("="*60)

fig5c, stats5c = create_brain_connectivity_plot(
    vertices=vertices,
    faces=faces,
    roi_coords_df=roi_coords_114_df,
    connectivity_matrix=conn_matrix,
    plot_title="Test 5c: PDF Export (Publication)",
    save_path=r"C:\Users\Azad Azargushasb\Research\HarrisLabPlotting\test_files\test5c_pdf_export.html",
    node_size=node_sizes_vector,
    edge_width=(0.5, 4.0),
    camera_view='anterior',
    node_metrics=metrics_df,
    # NEW: Export as PDF
    export_image=r"C:\Users\Azad Azargushasb\Research\HarrisLabPlotting\test_files\test5c_brain_network.pdf"
)

print("\nPDF export completed!")


TEST 5c: Export as PDF (publication format)
Creating brain connectivity visualization: Test 5c: PDF Export (Publication)
Connectivity matrix shape: (114, 114)
Node sizes: min=5.0, max=20.0
Loaded node metrics with columns: ['roi_index', 'node_idx', 'roi_name', 'module', 'participation_coef', 'within_module_zscore', 'node_role', 'pc_z_score', 'pc_p_value', 'pc_significant']
Added 114 nodes with valid coordinates
Added 452 edges above threshold 0.0
Node classification: 114 pos-only, 0 neg-only, 0 both
Saved interactive visualization to: C:\Users\Azad Azargushasb\Research\HarrisLabPlotting\test_files\test5c_pdf_export.html
Exporting PDF image...
Exported static image to: C:\Users\Azad Azargushasb\Research\HarrisLabPlotting\test_files\test5c_brain_network.pdf
  Format: PDF, Size: 223.9 KB

PDF export completed!


In [28]:
# Test 5d: High DPI export (removed custom dimensions - now fixed at 1200x900)
# Note: Scale is capped at 4x (~288 DPI) to avoid memory issues
print("\n" + "="*60)
print("TEST 5d: High DPI PNG Export")
print("="*60)

fig5d, stats5d = create_brain_connectivity_plot(
    vertices=vertices,
    faces=faces,
    roi_coords_df=roi_coords_114_df,
    connectivity_matrix=conn_matrix,
    plot_title="Test 5d: High DPI PNG",
    save_path=r"C:\Users\Azad Azargushasb\Research\HarrisLabPlotting\test_files\test5d_highres_export.html",
    node_size=node_sizes_vector,
    edge_width=(0.5, 4.0),
    camera_view='left',
    node_metrics=metrics_df,
    # High DPI export (fixed 1200x900 dimensions)
    export_image=r"C:\Users\Azad Azargushasb\Research\HarrisLabPlotting\test_files\test5d_brain_network_large.png",
    image_dpi=288  # 4x scale (max supported)
)

print("\nHigh DPI PNG export completed!")


TEST 5d: High DPI PNG Export
Creating brain connectivity visualization: Test 5d: High DPI PNG
Connectivity matrix shape: (114, 114)
Node sizes: min=5.0, max=20.0
Loaded node metrics with columns: ['roi_index', 'node_idx', 'roi_name', 'module', 'participation_coef', 'within_module_zscore', 'node_role', 'pc_z_score', 'pc_p_value', 'pc_significant']
Added 114 nodes with valid coordinates
Added 452 edges above threshold 0.0
Node classification: 114 pos-only, 0 neg-only, 0 both
Saved interactive visualization to: C:\Users\Azad Azargushasb\Research\HarrisLabPlotting\test_files\test5d_highres_export.html
Exporting PNG image...
Exported static image to: C:\Users\Azad Azargushasb\Research\HarrisLabPlotting\test_files\test5d_brain_network_large.png
  Format: PNG, Size: 1645.9 KB
  Dimensions: 4800x3600 pixels

High DPI PNG export completed!


## Summary

All tests completed. Check the generated files:

### Interactive HTML files:
- **test1**: Edge file input + pos/neg edge toggling with node hiding
- **test2**: Vector node sizes + metrics hover
- **test4**: Fixed edge width (no scaling)
- **test5a-5d**: Various image export tests (PNG, SVG, PDF)
- **test6a-6d**: Export display controls (title/legend visibility)
- **test7**: Node visibility when toggling edges

### Static image exports:
- **test5a_brain_network.png**: PNG at 300 DPI (publication quality)
- **test5b_brain_network.svg**: SVG vector format (infinitely scalable)
- **test5c_brain_network.pdf**: PDF for publications
- **test5d_brain_network_large.png**: High DPI PNG (288 DPI)
- **test6a_clean_export.png**: Clean PNG (no title, no legend)
- **test6b_title_only.png**: PNG with title, no legend
- **test6c_clean.svg**: Clean SVG (no title, no legend)
- **test6d_clean.pdf**: Clean PDF (no title, no legend)

### New Parameters for Image Export:
- `export_image`: Path to save static image (supports .png, .svg, .pdf)
- `image_format`: Format if not specified in path ('png', 'svg', 'pdf')
- `image_dpi`: Resolution (default 300 for publication quality)
- `export_show_title`: Whether to show title in export (default True)
- `export_show_legend`: Whether to show legend in export (default True)

### Removed Parameters:
- `image_width`: Removed - exports now use fixed 1200x900 dimensions
- `image_height`: Removed - exports now use fixed 1200x900 dimensions

### Bug Fixes:
- **Node visibility fix**: Nodes with BOTH positive and negative edges now properly hide when BOTH edge types are deselected in the legend

**Note**: Static images do NOT include camera controls dropdown or annotations for clean output.

In [29]:
# Test 6a: Export with NO title and NO legend (cleanest export)
print("\n" + "="*60)
print("TEST 6a: Export with no title, no legend (clean)")
print("="*60)

fig6a, stats6a = create_brain_connectivity_plot(
    vertices=vertices,
    faces=faces,
    roi_coords_df=new_roi_coords,
    connectivity_matrix=edge_file,
    plot_title="Test 6a: This title should NOT appear",
    save_path=r"C:\Users\Azad Azargushasb\Research\HarrisLabPlotting\test_files\test6a_no_title_no_legend.html",
    edge_width=(1.0, 6.0),
    camera_view='superior',
    # Export with title and legend hidden
    export_image=r"C:\Users\Azad Azargushasb\Research\HarrisLabPlotting\test_files\test6a_clean_export.png",
    export_show_title=False,
    export_show_legend=False,
    image_dpi=150
)

print("\nClean export (no title, no legend) completed!")


TEST 6a: Export with no title, no legend (clean)
Creating brain connectivity visualization: Test 6a: This title should NOT appear
Connectivity matrix shape: (28, 28)
Node sizes: min=8.0, max=8.0
Added 28 nodes with valid coordinates
Added 27 edges above threshold 0.0
Node classification: 18 pos-only, 7 neg-only, 3 both
Saved interactive visualization to: C:\Users\Azad Azargushasb\Research\HarrisLabPlotting\test_files\test6a_no_title_no_legend.html
Exporting PNG image...
Exported static image to: C:\Users\Azad Azargushasb\Research\HarrisLabPlotting\test_files\test6a_clean_export.png
  Format: PNG, Size: 429.9 KB
  Dimensions: 2500x1875 pixels

Clean export (no title, no legend) completed!


In [30]:
# Test 6b: Export with title only (no legend)
print("\n" + "="*60)
print("TEST 6b: Export with title, no legend")
print("="*60)

fig6b, stats6b = create_brain_connectivity_plot(
    vertices=vertices,
    faces=faces,
    roi_coords_df=new_roi_coords,
    connectivity_matrix=edge_file,
    plot_title="Brain Connectivity Network",
    save_path=r"C:\Users\Azad Azargushasb\Research\HarrisLabPlotting\test_files\test6b_title_only.html",
    edge_width=(1.0, 6.0),
    camera_view='oblique',
    # Export with title shown, legend hidden
    export_image=r"C:\Users\Azad Azargushasb\Research\HarrisLabPlotting\test_files\test6b_title_only.png",
    export_show_title=True,
    export_show_legend=False,
    image_dpi=150
)

print("\nTitle-only export completed!")


TEST 6b: Export with title, no legend
Creating brain connectivity visualization: Brain Connectivity Network
Connectivity matrix shape: (28, 28)
Node sizes: min=8.0, max=8.0
Added 28 nodes with valid coordinates
Added 27 edges above threshold 0.0
Node classification: 18 pos-only, 7 neg-only, 3 both
Saved interactive visualization to: C:\Users\Azad Azargushasb\Research\HarrisLabPlotting\test_files\test6b_title_only.html
Exporting PNG image...
Exported static image to: C:\Users\Azad Azargushasb\Research\HarrisLabPlotting\test_files\test6b_title_only.png
  Format: PNG, Size: 410.8 KB
  Dimensions: 2500x1875 pixels

Title-only export completed!


In [31]:
# Test 6c: Clean SVG export (no title, no legend)
print("\n" + "="*60)
print("TEST 6c: Clean SVG export (no title, no legend)")
print("="*60)

fig6c, stats6c = create_brain_connectivity_plot(
    vertices=vertices,
    faces=faces,
    roi_coords_df=new_roi_coords,
    connectivity_matrix=edge_file,
    plot_title="This should not appear",
    save_path=r"C:\Users\Azad Azargushasb\Research\HarrisLabPlotting\test_files\test6c_clean_svg.html",
    edge_width=(1.0, 6.0),
    camera_view='anterior',
    # Clean SVG export
    export_image=r"C:\Users\Azad Azargushasb\Research\HarrisLabPlotting\test_files\test6c_clean.svg",
    export_show_title=False,
    export_show_legend=False
)

print("\nClean SVG export completed!")


TEST 6c: Clean SVG export (no title, no legend)
Creating brain connectivity visualization: This should not appear
Connectivity matrix shape: (28, 28)
Node sizes: min=8.0, max=8.0
Added 28 nodes with valid coordinates
Added 27 edges above threshold 0.0
Node classification: 18 pos-only, 7 neg-only, 3 both
Saved interactive visualization to: C:\Users\Azad Azargushasb\Research\HarrisLabPlotting\test_files\test6c_clean_svg.html
Exporting SVG image...
Exported static image to: C:\Users\Azad Azargushasb\Research\HarrisLabPlotting\test_files\test6c_clean.svg
  Format: SVG, Size: 309.8 KB

Clean SVG export completed!


In [32]:
# Test 6d: Clean PDF export (no title, no legend) - ideal for publications
print("\n" + "="*60)
print("TEST 6d: Clean PDF export (no title, no legend)")
print("="*60)

fig6d, stats6d = create_brain_connectivity_plot(
    vertices=vertices,
    faces=faces,
    roi_coords_df=new_roi_coords,
    connectivity_matrix=edge_file,
    plot_title="This should not appear",
    save_path=r"C:\Users\Azad Azargushasb\Research\HarrisLabPlotting\test_files\test6d_clean_pdf.html",
    edge_width=(1.0, 6.0),
    camera_view='left',
    # Clean PDF export for publications
    export_image=r"C:\Users\Azad Azargushasb\Research\HarrisLabPlotting\test_files\test6d_clean.pdf",
    export_show_title=False,
    export_show_legend=False
)

print("\nClean PDF export completed!")


TEST 6d: Clean PDF export (no title, no legend)
Creating brain connectivity visualization: This should not appear
Connectivity matrix shape: (28, 28)
Node sizes: min=8.0, max=8.0
Added 28 nodes with valid coordinates
Added 27 edges above threshold 0.0
Node classification: 18 pos-only, 7 neg-only, 3 both
Saved interactive visualization to: C:\Users\Azad Azargushasb\Research\HarrisLabPlotting\test_files\test6d_clean_pdf.html
Exporting PDF image...
Exported static image to: C:\Users\Azad Azargushasb\Research\HarrisLabPlotting\test_files\test6d_clean.pdf
  Format: PDF, Size: 98.8 KB

Clean PDF export completed!


## Test 7: Node Visibility with Edge Toggling

Test that nodes with BOTH positive and negative edges properly hide when BOTH edge types are deselected in the legend.

**Expected behavior:**
- When only positive edges shown → nodes with positive edges visible (including nodes with both)
- When only negative edges shown → nodes with negative edges visible (including nodes with both)  
- When BOTH edge types hidden → ALL nodes should hide (brain surface only)

In [33]:
# Test 7: Node visibility with edge toggling
# This test uses data with BOTH positive and negative edges to test the fix
print("\n" + "="*60)
print("TEST 7: Node visibility when toggling edges")
print("="*60)

# Verify we have nodes with both edge types
edge_matrix = load_edge_file(edge_file)
print(f"Positive edges: {np.sum(edge_matrix > 0)}")
print(f"Negative edges: {np.sum(edge_matrix < 0)}")

fig7, stats7 = create_brain_connectivity_plot(
    vertices=vertices,
    faces=faces,
    roi_coords_df=new_roi_coords,
    connectivity_matrix=edge_file,
    plot_title="Test 7: Node Visibility with Edge Toggle",
    save_path=r"C:\Users\Azad Azargushasb\Research\HarrisLabPlotting\test_files\test7_node_visibility.html",
    edge_width=(1.0, 8.0),
    camera_view='superior',
    hide_nodes_with_hidden_edges=True,  # This enables the node-edge linking
    node_size=12,
    label_font_size=14
)

print(f"\nNodes with positive edges only: {stats7['nodes_with_pos_only']}")
print(f"Nodes with negative edges only: {stats7['nodes_with_neg_only']}")
print(f"Nodes with BOTH edge types: {stats7['nodes_with_both']}")
print("\n*** IMPORTANT: Open the HTML file and try clicking the legend items ***")
print("*** When BOTH 'Positive Edges' and 'Negative Edges' are hidden, ***")
print("*** ALL nodes should disappear, leaving only the brain surface ***")


TEST 7: Node visibility when toggling edges
Positive edges: 36
Negative edges: 18
Creating brain connectivity visualization: Test 7: Node Visibility with Edge Toggle
Connectivity matrix shape: (28, 28)
Node sizes: min=12.0, max=12.0
Added 28 nodes with valid coordinates
Added 27 edges above threshold 0.0
Node classification: 18 pos-only, 7 neg-only, 3 both
Saved interactive visualization to: C:\Users\Azad Azargushasb\Research\HarrisLabPlotting\test_files\test7_node_visibility.html

Nodes with positive edges only: 18
Nodes with negative edges only: 7
Nodes with BOTH edge types: 3

*** IMPORTANT: Open the HTML file and try clicking the legend items ***
*** When BOTH 'Positive Edges' and 'Negative Edges' are hidden, ***
*** ALL nodes should disappear, leaving only the brain surface ***


In [34]:
# Display figure - try clicking on legend items to toggle visibility
fig7.show()

## Summary

All tests completed. Check the generated files:

### Interactive HTML files:
- **test1**: Edge file input + pos/neg edge toggling with node hiding
- **test2**: Vector node sizes + metrics hover
- **test4**: Fixed edge width (no scaling)
- **test5a-5d**: Various image export tests

### Static image exports:
- **test5a_brain_network.png**: PNG at 300 DPI (publication quality)
- **test5b_brain_network.svg**: SVG vector format (infinitely scalable)
- **test5c_brain_network.pdf**: PDF for publications
- **test5d_brain_network_highres.png**: High-res PNG (600 DPI, 1920x1080)

### New Parameters for Image Export:
- `export_image`: Path to save static image (supports .png, .svg, .pdf)
- `image_format`: Format if not specified in path ('png', 'svg', 'pdf')
- `image_dpi`: Resolution (default 300 for publication quality)
- `image_width`: Width in pixels (default 1200)
- `image_height`: Height in pixels (default 900)

**Note**: Static images do NOT include camera controls annotation for clean output.

## Test 8: Node Color Vector Feature

Test the new `node_color` parameter that accepts:
1. **Single color string** (backwards compatible)
2. **Module assignments array** (integers) - auto-generates colors
3. **Color strings array** - per-node colors
4. **CSV file path** - loads module assignments from file

In [35]:
# IMPORTANT: Reload the module to get the latest changes
import importlib
import HarrisLabPlotting
import HarrisLabPlotting.connectivity
import HarrisLabPlotting.utils

# Reload modules
importlib.reload(HarrisLabPlotting.utils)
importlib.reload(HarrisLabPlotting.connectivity)

# Re-import functions
from HarrisLabPlotting import (
    create_brain_connectivity_plot,
    create_brain_connectivity_plot_with_modularity,
    convert_node_color_input,
    generate_module_colors
)

print("Module reloaded with new node_color features!")

Module reloaded with new node_color features!


In [36]:
# Test data paths for modularity tests
module_assignments_file = r"C:\Users\Azad Azargushasb\Research\HarrisLabPlotting\test_files\k5_data\state_0\module_assignments.csv"
conn_matrix_file = r"C:\Users\Azad Azargushasb\Research\HarrisLabPlotting\test_files\k5_data\state_0\connectivity_matrix.csv"

# Load and preview module assignments
print("Loading module assignments...")
module_df = pd.read_csv(module_assignments_file)
print(f"Module assignments: {len(module_df)} rows")
print(f"Columns: {module_df.columns.tolist()}")
print(f"\nUnique modules: {module_df['module'].unique()}")
print(f"Module distribution:")
print(module_df['module'].value_counts().sort_index())

Loading module assignments...
Module assignments: 114 rows
Columns: ['roi_index', 'module']

Unique modules: [1 2 3 4 5 6]
Module distribution:
module
1    19
2    31
3    28
4    18
5     6
6    12
Name: count, dtype: int64


In [37]:
# Test 8a: Node color using module assignments array (integers)
print("\n" + "="*60)
print("TEST 8a: Node color from module assignments array")
print("="*60)

# Get module assignments as numpy array
module_array = module_df['module'].values
print(f"Module array shape: {module_array.shape}")
print(f"Module values range: {module_array.min()} to {module_array.max()}")

fig8a, stats8a = create_brain_connectivity_plot(
    vertices=vertices,
    faces=faces,
    roi_coords_df=roi_coords_114_df,
    connectivity_matrix=conn_matrix_file,
    plot_title="Test 8a: Node Colors from Module Assignments",
    save_path=r"C:\Users\Azad Azargushasb\Research\HarrisLabPlotting\test_files\test8a_module_colors.html",
    node_size=10,
    node_color=module_array,  # Pass module assignments directly!
    node_border_color='darkgray',
    edge_width=(0.5, 4.0),
    camera_view='oblique'
)

print(f"\nNode color is vector: {stats8a['node_color_is_vector']}")
print(f"Module color map: {stats8a['module_color_map']}")


TEST 8a: Node color from module assignments array
Module array shape: (114,)
Module values range: 1 to 6
Creating brain connectivity visualization: Test 8a: Node Colors from Module Assignments
Connectivity matrix shape: (114, 114)
Node sizes: min=10.0, max=10.0
Node colors: 6 unique colors
Added 114 nodes with valid coordinates
Added 452 edges above threshold 0.0
Node classification: 114 pos-only, 0 neg-only, 0 both
Saved interactive visualization to: C:\Users\Azad Azargushasb\Research\HarrisLabPlotting\test_files\test8a_module_colors.html

Node color is vector: True
Module color map: {1: 'rgb(227, 26, 28)', 2: 'rgb(51, 160, 44)', 3: 'rgb(31, 120, 180)', 4: 'rgb(255, 127, 0)', 5: 'rgb(106, 61, 154)', 6: 'rgb(177, 89, 40)'}


In [38]:
# Display figure
fig8a.show()

In [39]:
# Test 8b: Node color from CSV file path
print("\n" + "="*60)
print("TEST 8b: Node color from CSV file path")
print("="*60)

fig8b, stats8b = create_brain_connectivity_plot(
    vertices=vertices,
    faces=faces,
    roi_coords_df=roi_coords_114_df,
    connectivity_matrix=conn_matrix_file,
    plot_title="Test 8b: Node Colors from CSV File",
    save_path=r"C:\Users\Azad Azargushasb\Research\HarrisLabPlotting\test_files\test8b_module_colors_csv.html",
    node_size=10,
    node_color=module_assignments_file,  # Pass CSV file path!
    node_border_color='darkgray',
    edge_width=(0.5, 4.0),
    camera_view='anterior'
)

print(f"\nNode color is vector: {stats8b['node_color_is_vector']}")
print(f"Module color map: {stats8b['module_color_map']}")


TEST 8b: Node color from CSV file path
Creating brain connectivity visualization: Test 8b: Node Colors from CSV File
Connectivity matrix shape: (114, 114)
Node sizes: min=10.0, max=10.0
Node colors: 6 unique colors
Added 114 nodes with valid coordinates
Added 452 edges above threshold 0.0
Node classification: 114 pos-only, 0 neg-only, 0 both
Saved interactive visualization to: C:\Users\Azad Azargushasb\Research\HarrisLabPlotting\test_files\test8b_module_colors_csv.html

Node color is vector: True
Module color map: {1: 'rgb(227, 26, 28)', 2: 'rgb(51, 160, 44)', 3: 'rgb(31, 120, 180)', 4: 'rgb(255, 127, 0)', 5: 'rgb(106, 61, 154)', 6: 'rgb(177, 89, 40)'}


## Test 9: Modularity Visualization Function

Test the new `create_brain_connectivity_plot_with_modularity()` function which:
1. Takes module assignments as a separate parameter
2. Auto-generates module colors with a legend
3. Optionally shows Q and Z scores in the title

In [40]:
# Test 9a: Basic modularity visualization with Q and Z scores
print("\n" + "="*60)
print("TEST 9a: Modularity visualization with Q and Z scores")
print("="*60)

fig9a, stats9a = create_brain_connectivity_plot_with_modularity(
    vertices=vertices,
    faces=faces,
    roi_coords_df=roi_coords_114_df,
    connectivity_matrix=conn_matrix_file,
    module_assignments=module_assignments_file,  # CSV file path
    plot_title="Brain Network Modularity",
    save_path=r"C:\Users\Azad Azargushasb\Research\HarrisLabPlotting\test_files\test9a_modularity_viz.html",
    Q_score=0.452,  # Example Q score
    Z_score=3.21,   # Example Z score
    node_size=10,
    edge_width=(0.5, 4.0),
    camera_view='oblique'
)

print(f"\nNumber of modules: {stats9a['n_modules']}")
print(f"Module sizes: {stats9a['module_sizes']}")
print(f"Q score in stats: {stats9a['Q_score']}")
print(f"Z score in stats: {stats9a['Z_score']}")


TEST 9a: Modularity visualization with Q and Z scores
Creating modularity visualization: Brain Network Modularity
Creating brain connectivity visualization: Brain Network Modularity (Q=0.452, Z=3.21)
Connectivity matrix shape: (114, 114)
Node sizes: min=10.0, max=10.0
Node colors: 6 unique colors
Added 114 nodes with valid coordinates
Added 452 edges above threshold 0.0
Node classification: 114 pos-only, 0 neg-only, 0 both
Saved interactive visualization to: C:\Users\Azad Azargushasb\Research\HarrisLabPlotting\test_files\test9a_modularity_viz.html

Number of modules: 6
Module sizes: {'module_1': 19, 'module_2': 31, 'module_3': 28, 'module_4': 18, 'module_5': 6, 'module_6': 12}
Q score in stats: 0.452
Z score in stats: 3.21


In [41]:
# Display the modularity visualization
fig9a.show()

## Summary: New Node Color and Modularity Features

### New `node_color` parameter in `create_brain_connectivity_plot()`:
- **Single color string** (backwards compatible): `node_color='purple'`
- **Module assignments array** (integers, 1-indexed): `node_color=module_array`
- **CSV file path**: `node_color='path/to/modules.csv'`
- Auto-generates visually distinct colors for each module
- Returns `module_color_map` in stats dict

### New `create_brain_connectivity_plot_with_modularity()` function:
- Takes `module_assignments` as a separate parameter
- Creates module legend in the plot
- Optional `Q_score` and `Z_score` parameters for title
- **`edge_color_mode` parameter**:
  - `'sign'` (default): Edges colored red/blue by positive/negative sign
  - `'module'`: Edges colored by the module of the source node
- Returns module statistics including sizes and assignments

### Test Files Generated:
- **test8a_module_colors.html**: Node colors from module array
- **test8b_module_colors_csv.html**: Node colors from CSV file
- **test9a_modularity_viz.html**: Full modularity viz with Q/Z scores (sign-colored edges)
- **test9b_modularity_no_scores.html**: Modularity viz without scores
- **test9c_module_edges.html**: Modularity viz with module-colored edges

### Removed Function:
- `create_simple_modularity_visualization()` - replaced by new functionality

## Summary: New Node Color and Modularity Features

### New `node_color` parameter in `create_brain_connectivity_plot()`:
- **Single color string** (backwards compatible): `node_color='purple'`
- **Module assignments array** (integers, 1-indexed): `node_color=module_array`
- **CSV file path**: `node_color='path/to/modules.csv'`
- Auto-generates visually distinct colors for each module
- Returns `module_color_map` in stats dict

### New `create_brain_connectivity_plot_with_modularity()` function:
- Takes `module_assignments` as a separate parameter
- Creates module legend in the plot
- Optional `Q_score` and `Z_score` parameters for title
- Returns module statistics including sizes and assignments

### Test Files Generated:
- **test8a_module_colors.html**: Node colors from module array
- **test8b_module_colors_csv.html**: Node colors from CSV file
- **test9a_modularity_viz.html**: Full modularity viz with Q/Z scores
- **test9b_modularity_no_scores.html**: Modularity viz without scores

### Removed Function:
- `create_simple_modularity_visualization()` - replaced by new functionality

In [42]:
# Test 9c: Modularity visualization with MODULE-colored edges
print("\n" + "="*60)
print("TEST 9c: Modularity visualization with module-colored edges")
print("="*60)

# Reload to get latest changes
importlib.reload(HarrisLabPlotting.connectivity)
from HarrisLabPlotting.connectivity import create_brain_connectivity_plot_with_modularity

fig9c, stats9c = create_brain_connectivity_plot_with_modularity(
    vertices=vertices,
    faces=faces,
    roi_coords_df=roi_coords_114_df,
    connectivity_matrix=conn_matrix_file,
    module_assignments=module_assignments_file,
    plot_title="Modularity with Module-Colored Edges",
    save_path=r"C:\Users\Azad Azargushasb\Research\HarrisLabPlotting\test_files\test9c_module_edges.html",
    Q_score=0.452,
    Z_score=3.21,
    edge_color_mode='module',  # NEW: Color edges by module instead of pos/neg
    node_size=10,
    edge_width=(0.5, 4.0),
    camera_view='oblique'
)

print(f"\nEdge color mode: {stats9c['edge_color_mode']}")
print(f"Number of modules: {stats9c['n_modules']}")


TEST 9c: Modularity visualization with module-colored edges
Creating modularity visualization: Modularity with Module-Colored Edges
Creating brain connectivity visualization: Modularity with Module-Colored Edges (Q=0.452, Z=3.21)
Connectivity matrix shape: (114, 114)
Node sizes: min=10.0, max=10.0
Node colors: 6 unique colors
Added 114 nodes with valid coordinates
Added 452 edges above threshold 0.0
Node classification: 114 pos-only, 0 neg-only, 0 both
Saved interactive visualization to: C:\Users\Azad Azargushasb\Research\HarrisLabPlotting\test_files\test9c_module_edges.html

Edge color mode: module
Number of modules: 6


In [43]:
# Display the module-colored edges visualization
fig9c.show()